In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms 

In [2]:
num_epochs=5
batch_size=64
learning_rate=0.001
num_classes=10

In [3]:
train_dataset=torchvision.datasets.MNIST("../../data",train=True,transform=transforms.ToTensor(),download=False)
test_dataset=torchvision.datasets.MNIST("../../data",train=False,transform=transforms.ToTensor(),download=False)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
class ConvNet(nn.Module):
    def __init__ (self,num_classes):
        super().__init__()
        self.layer1=nn.Sequential(nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
                                 nn.BatchNorm2d(16),
                                 nn.ReLU(),
                                 nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer2=nn.Sequential(nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.MaxPool2d(kernel_size=2,stride=2))
        self.fc=nn.Linear(7*7*32,num_classes)
    
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [5]:
model=ConvNet(num_classes)
model.train()
loss_func=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=learning_rate)
print("epoch,\tstep,\tloss")

for epoch in range(num_epochs):
    for i ,(x,y) in enumerate(train_loader):
        y_hat=model(x)
        loss=loss_func(y_hat,y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if (i+1) % 300 ==0:
            print("{},\t{},\t{:.4f}".format(epoch+1,i+1,loss))

epoch,	step,	loss


C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0,	300,	0.0992
0,	600,	0.0467
0,	900,	0.1268
1,	300,	0.0225
1,	600,	0.0720
1,	900,	0.0425
2,	300,	0.0188
2,	600,	0.1730
2,	900,	0.0140
3,	300,	0.0067
3,	600,	0.0017
3,	900,	0.0272
4,	300,	0.0046
4,	600,	0.0133
4,	900,	0.1397


In [7]:
for para in model.parameters():
    print(para)

Parameter containing:
tensor([[[[ 0.0565,  0.0360,  0.0945,  0.0123, -0.1736],
          [ 0.2299,  0.1250,  0.0527, -0.0995, -0.0988],
          [ 0.0768, -0.1285, -0.1606, -0.1793,  0.0386],
          [-0.0875, -0.1214, -0.1374, -0.0962,  0.1744],
          [-0.1617, -0.1151, -0.0976,  0.1008,  0.1106]]],


        [[[-0.0325, -0.1902, -0.1332, -0.0214,  0.2159],
          [ 0.1012,  0.0818,  0.1585, -0.0336, -0.2118],
          [ 0.1147, -0.1363, -0.0205,  0.0694, -0.1763],
          [-0.1283,  0.1826, -0.0267, -0.0739, -0.0933],
          [-0.1180, -0.2093,  0.0210,  0.0723,  0.2151]]],


        [[[-0.0890, -0.2360, -0.0632,  0.0947, -0.1777],
          [-0.2670,  0.1748, -0.0945,  0.0923,  0.0714],
          [ 0.1964, -0.1164, -0.1200, -0.1346, -0.1286],
          [ 0.2427,  0.1175,  0.0595, -0.0019, -0.1781],
          [ 0.0078,  0.0252, -0.1594, -0.1666,  0.0226]]],


        [[[ 0.0807, -0.0157, -0.0222,  0.0100, -0.2725],
          [-0.0146,  0.2141, -0.0620, -0.0081, -0.3100

Parameter containing:
tensor([[ 0.0085, -0.0221, -0.0582,  ..., -0.2178, -0.1730,  0.0113],
        [ 0.0453,  0.0042,  0.0401,  ...,  0.0134, -0.0136,  0.0362],
        [-0.0095,  0.0255,  0.0411,  ..., -0.0712, -0.0886,  0.1247],
        ...,
        [ 0.0149, -0.0033, -0.0005,  ...,  0.0646, -0.0171, -0.0259],
        [-0.0031,  0.0273,  0.0119,  ..., -0.0510, -0.0242, -0.0707],
        [ 0.0049, -0.0205, -0.0144,  ...,  0.0148,  0.0490, -0.0607]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0121,  0.0672, -0.0205, -0.0395, -0.0119,  0.0224, -0.0024, -0.0013,
        -0.0128, -0.0307], requires_grad=True)


In [8]:
with torch.no_grad():
    for images,labels in test_loader:
        outputs=model(images)

In [9]:
for para in model.parameters():
    print(para)

Parameter containing:
tensor([[[[ 0.0565,  0.0360,  0.0945,  0.0123, -0.1736],
          [ 0.2299,  0.1250,  0.0527, -0.0995, -0.0988],
          [ 0.0768, -0.1285, -0.1606, -0.1793,  0.0386],
          [-0.0875, -0.1214, -0.1374, -0.0962,  0.1744],
          [-0.1617, -0.1151, -0.0976,  0.1008,  0.1106]]],


        [[[-0.0325, -0.1902, -0.1332, -0.0214,  0.2159],
          [ 0.1012,  0.0818,  0.1585, -0.0336, -0.2118],
          [ 0.1147, -0.1363, -0.0205,  0.0694, -0.1763],
          [-0.1283,  0.1826, -0.0267, -0.0739, -0.0933],
          [-0.1180, -0.2093,  0.0210,  0.0723,  0.2151]]],


        [[[-0.0890, -0.2360, -0.0632,  0.0947, -0.1777],
          [-0.2670,  0.1748, -0.0945,  0.0923,  0.0714],
          [ 0.1964, -0.1164, -0.1200, -0.1346, -0.1286],
          [ 0.2427,  0.1175,  0.0595, -0.0019, -0.1781],
          [ 0.0078,  0.0252, -0.1594, -0.1666,  0.0226]]],


        [[[ 0.0807, -0.0157, -0.0222,  0.0100, -0.2725],
          [-0.0146,  0.2141, -0.0620, -0.0081, -0.3100